In [1]:

from keras.models import Model, Sequential
from keras.layers import *
import numpy as np
import keras.backend as K
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
K.set_image_dim_ordering('tf')
from keras.callbacks import ModelCheckpoint

from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda, Dropout, merge
import numpy as np
import keras.backend as K
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.utils import to_categorical
from keras.layers import concatenate,Conv2DTranspose
from keras.preprocessing.image import ImageDataGenerator
K.set_image_dim_ordering('tf')
from keras.callbacks import ModelCheckpoint
train = np.load('/home/shubham/Documents/CrackForest-dataset-master/train.npy')
labels = np.load('/home/shubham/Documents/CrackForest-dataset-master/labelsBound.npy')
#labels = to_categorical(labels)
labels = labels.reshape(labels.shape[0],320,480,1)
print np.unique(labels)
#labels = to_categorical(y=labels,n)
import tensorflow as tf
print labels.shape
print train.shape
smooth = 1.
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=1)
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def bce_dice(y_true, y_pred):
    return -(binary_crossentropy(y_true, y_pred)+(dice_coef(y_true, y_pred)))

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
inputs = Input((320,480,3))
x = Conv2D(filters=32,activation='relu',dilation_rate=(1,1),kernel_size=(3,3),padding='same')(inputs)
x = Conv2D(filters=32,activation='relu',dilation_rate=(1,1),kernel_size=(3,3),padding='same')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(filters=64,activation='relu',dilation_rate=(2,2),kernel_size=(3,3),padding='same')(x)
x = Conv2D(filters=64,activation='relu',dilation_rate=(2,2),kernel_size=(3,3),padding='same')(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Conv2D(filters=128,activation='relu',dilation_rate=(3,3),kernel_size=(3,3),padding='same')(x)
x = Conv2D(filters=128,activation='relu',dilation_rate=(3,3),kernel_size=(3,3),padding='same')(x)
x = Conv2D(filters=128,activation='relu',dilation_rate=(3,3),kernel_size=(3,3),padding='same')(x)
x = Conv2D(filters=128,activation='relu',dilation_rate=(3,3),kernel_size=(3,3),padding='same')(x)
x = Conv2D(filters=128,activation='relu',dilation_rate=(2,2),kernel_size=(3,3),padding='same')(x)
x = Conv2D(filters=128,activation='relu',dilation_rate=(2,2),kernel_size=(3,3),padding='same')(x)
x = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same',kernel_initializer='he_normal')(x)
x = Conv2D(filters=128,activation='relu',dilation_rate=(1,1),kernel_size=(3,3),padding='same')(x)
x = Conv2D(filters=128,activation='relu',dilation_rate=(1,1),kernel_size=(3,3),padding='same')(x)
x = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same',kernel_initializer='he_normal')(x)
x = Conv2D(filters=32,activation='relu',dilation_rate=(3,3),kernel_size=(7,7),padding='same')(x)
x = Conv2D(filters=32,activation='relu',dilation_rate=(1,1),kernel_size=(1,1),padding='same')(x)
x = Conv2D(filters=1,activation='sigmoid',kernel_size=(1,1),padding='same')(x)

model = Model(inputs=inputs,outputs=x)

/home/shubham/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[0 1]
(118, 320, 480, 1)
(118, 320, 480, 3)


In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 320, 480, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 320, 480, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 320, 480, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 160, 240, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 160, 240, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 160, 240, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 80, 120, 64)       0         
__________

In [ ]:
model.compile(optimizer=Adam(lr=1e-5,decay=1e-6), loss=bce_dice, metrics=[dice_coef])

In [ ]:
checkpoint = ModelCheckpoint(filepath='/home/shubham/Documents/CrackForest-dataset-master/mynet1.h5',
                             monitor='val_loss',verbose=1,save_best_only=True)
#model.load_weights('/home/shubham/Documents/CrackForest-dataset-master/mynet.h5')
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90.,
                     zoom_range=0.2,shear_range=0.2)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train, augment=True, seed=seed)


# combine generators into one which yields image and masks
train_generator = zip(image_datagen.flow(x=train[:100],batch_size=1,seed=1), mask_datagen.flow(x=labels[:100],batch_size=1,seed=1))
valid_generator = zip(image_datagen.flow(x=train[100:],batch_size=1,seed=1), mask_datagen.flow(x=labels[100:],batch_size=1,seed=1))
model.fit_generator(callbacks=[checkpoint],validation_data=valid_generator,validation_steps=40,
    generator=train_generator,
    steps_per_epoch=100,
    epochs=100)

Epoch 1/100


/home/shubham/miniconda2/lib/python2.7/site-packages/keras/preprocessing/image.py:594: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/shubham/miniconda2/lib/python2.7/site-packages/keras/preprocessing/image.py:602: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


 14/100 [===>..........................] - ETA: 2:37 - loss: -0.7131 - dice_coef: 0.0193

In [ ]:
from skimage import io


In [ ]:
img = io.imread('/home/shubham/Documents/CrackForest-dataset-master/seg/')

In [ ]:
io.imshow(img)

In [ ]:
img  = img/255.
img = (img-np.mean(img))/np.std(img)

In [ ]:
img.shape

In [ ]:
img = img.reshape(1,320,480,3)
out = model.predict(img)
print out.shape
out = out.transpose([0,3,1,2])
io.imshow(out[0][0])

In [ ]:
model.save_weights('/home/shubham/Documents/CrackForest-dataset-master/mymodel.h5')